
# Hollow Glass Ring in Air

N1 in Bird

In [ ]:
import numpy as np

from fiberamp.fiber.microstruct.bragg import BraggScalar, Bragg
from fiberamp.fiber.microstruct.bragg import plotlogf
from ngsolve.webgui import Draw
from ngsolve import CF
from scipy.optimize import newton
from scipy.linalg import null_space

In [ ]:
n_air = 1
n_glass = 1.44

ns = [lambda x:n_air, lambda x:n_glass, lambda x:n_air]

ts = [5e-05, 1e-5, 5e-05]

mats = ['air', 'glass', 'air']

maxhs = [.1,.01, .1]
wl = 1.2e-06
scale = ts[0]

In [ ]:
A = BraggScalar(ts=ts, mats=mats, ns=ns, maxhs=maxhs, wl=wl, scale=scale)


In [ ]:
Draw(A.mesh)

In [ ]:
k_low = A.k0 * A.ns[0] * A.scale
k_high = A.k0 * A.ns[1] * A.scale
k_low, k_high

In [ ]:
outer = 'h2'
Ktype = 'kappa'
nu = 0

In [ ]:
plotlogf(A.determinant,.9999*k_low,1.00001*k_low, -.01,.01, nu, outer, Ktype,
         iref=100, rref=100, levels=100, figsize=(10,6))

In [ ]:
guess = np.array(.99995*k_low)

beta1 = newton(A.determinant, guess, args=(nu, outer, Ktype), tol = 1e-15)

print("Scaled beta: ", beta1, ". Residual of determinant: ", abs(A.determinant(beta1, nu, outer, Ktype)))


# Bragg Scalar Function

In [ ]:
U = A.all_fields(beta1, nu, outer, Ktype)

In [ ]:
Draw(1e1*U, A.mesh, min=-3, max=10)

### 1D

In [ ]:
u = A.fields_matplot(beta1, nu, outer, Ktype)

In [ ]:
%matplotlib notebook
fig, ax = A.plot1D(u['Ez_rad'], double_r=True, rlist=[400,10000,400], nu=nu, maxscale=True,
                  linewidth=1.5, color='k')
Rhos = A.rhos/A.scale

ax.arrow(0,0, Rhos[-1], 0, color='lightgray', width=.001, 
         head_width=.03, head_length=.1)
ax.arrow(0,0, -Rhos[-1], 0, color='lightgray', width=.001, 
         head_width=.03, head_length=.1)

ax.set_frame_on(False)
ax.set_xticks([])
ax.set_yticks([])
fig.tight_layout()
m, M = ax.get_ylim()
# for r in Rhos:
#     ax.plot([r,r], [m, M], 'orange', linewidth=1.2, linestyle=':')
#     ax.plot([-r,-r], [m, M], 'orange', linewidth=1.2, linestyle=':')
ax.set_xlabel('\nDistance from Fiber Center')
fig.savefig('/home/pv/papers/outer_materials/figures/images/jpgs/1dlong.jpg')

# Numerical

In [ ]:
ns = [lambda x:n_air, lambda x:n_glass, lambda x:n_air,  lambda x:n_air]

ts = [5e-05, 1e-5, 2e-05, 2e-05]

mats = ['air', 'glass', 'air', 'Outer']

maxhs = [.1,.01, .08, .1]
wl=1.2e-06
scale = ts[0]
B = Bragg(ts=ts, mats=mats, ns=ns, maxhs=maxhs, wl=wl, scale=scale)


In [ ]:
Draw(B.mesh)

In [ ]:
Z2 = B.sqrZfrom(beta1/A.scale)
Z = Z2**.5
Z

In [ ]:
center = Z
radius = 0.1

p = 1
z, y, yl, beta, P, _ = B.leakymode(p,nspan=4,npts=4,
                                    rad=radius,
                                    ctr=center,
#                                     rhoinv=.9,
#                                     quadrule='ellipse_trapez_shift',
                                    alpha=5,
                                    niterations=5, nrestarts=0)

In [ ]:
for f in y:
    Draw(1e3*f, B.mesh)

# Resonant mode ?

Would be good to do the wavelength study on this and see if it still has spikes

In [ ]:
# Formula for loss spikes from article

ms = np.arange(11,15,1)

n1 = A.ns[0]  # Inner (core) index
n2 = A.ns[1]  # Cladding index

d = 15*2/3*1e-6

ls = (2 * n1 * d / ms * ((n2/n1)**2 - 1)**.5)  # when n2 depends on ls, need solver
ls

In [ ]:
wl_res= 1.592e-06

In [ ]:
n_air = 1.00027717
n_glass = 1.4388164768221814

ns = [lambda x:n_air, lambda x:n_glass, lambda x:n_air]

ts = [4.0775e-05, 1e-5, 4.0775e-05]

mats = ['air', 'glass', 'air']

maxhs = [.1,.015, .1]

In [ ]:
B = BraggExact(ts=ts, mats=mats, ns=ns, maxhs=maxhs, wl=wl_res)



In [ ]:
k_low = B.k0 * B.ns[0] * B.scale
k_low

In [ ]:
outer = 'h2'
nu = 1

In [ ]:
plotlogf(B.determinant, .995*k_low, 1.001*k_low, -.1,.1, nu, outer,
         iref=100, rref=100, levels=100)

In [ ]:
guess = np.array(.9999 * k_low)

beta1 = newton(B.determinant, guess, args=(nu, outer), tol = 1e-15)

print("Scaled beta: ", beta1, ". Residual of determinant: ", abs(B.determinant(beta1, nu, outer)))


# Bragg Exact Fields

In [ ]:
FsB = B.all_fields(beta1, nu, outer)

### Electric

In [ ]:
Draw(FsB['Ez'], B.mesh)

In [ ]:
fsB = B.fields_matplot(beta1, nu, outer)

In [ ]:
%matplotlib notebook
figsize=(12,6)
fig, ax = B.plot1D(fsB['Sz_rad'], double_r=True, rlist=[400,1000,400], nu=nu, 
                   part='norm', maxscale=True, figsize=figsize, linewidth=.6)
B.add1D_plot(ax, fsA['Sz_rad'], double_r=True,  nu=nu, part='norm', maxscale=True, linewidth=.6)

In [ ]:
Draw(FsB['Sphi'], A.mesh)